# Title

In [230]:
import numpy as np
import pandas as pd
import nltk
nltk.download('stopwords')
nltk.download('words')
import os
import re
import sqlalchemy

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\brand\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\brand\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


## Dataset

In [231]:
CWD = os.getcwd()
dataset_engine = sqlalchemy.create_engine(f"sqlite:///{CWD}/modeling_dataset.db")
dataset_engine.connect()

In [232]:
tables = pd.read_sql("SELECT * FROM sqlite_master WHERE type = 'table'", dataset_engine)
tables

,type,name,tbl_name,rootpage,sql
0,table,by,by,2,"CREATE TABLE ""by"" (\n\t""index"" BIGINT, \n\tlic..."
1,table,by-sa,by-sa,28,"CREATE TABLE ""by-sa"" (\n\t""index"" BIGINT, \n\t..."
2,table,by-nc,by-nc,410,"CREATE TABLE ""by-nc"" (\n\t""index"" BIGINT, \n\t..."
3,table,by-nc-sa,by-nc-sa,436,"CREATE TABLE ""by-nc-sa"" (\n\t""index"" BIGINT, \..."
4,table,by-nd,by-nd,1223,"CREATE TABLE ""by-nd"" (\n\t""index"" BIGINT, \n\t..."
5,table,by-nc-nd,by-nc-nd,1590,"CREATE TABLE ""by-nc-nd"" (\n\t""index"" BIGINT, \..."
6,table,publicdomain,publicdomain,803,"CREATE TABLE publicdomain (\n\t""index"" BIGINT,..."


In [233]:
webpages_dataset = pd.concat(
    [
        pd.read_sql(f"SELECT * FROM '{table_name}'", dataset_engine)
        for table_name in tables["name"]
    ]
)
webpages_dataset = webpages_dataset\
    .loc[webpages_dataset["contents"] != "", :]\
    .reset_index()\
    .drop(["index", "title", "level_0"], axis=1)
webpages_dataset.sample(5)

,license,url,contents
3294,licenses/by-nc/1.0,https://serc.carleton.edu/details/images/31960...,figure 1.0 figure 1.0 View Original Image at F...
3763,licenses/by-nc/2.0,https://guides.library.queensu.ca/c.php?g=7047...,How to Apply a Creative Commons License to You...
7095,licenses/by-nc-nd/3.0,https://www.compadre.org/osp/bulletinboard/TDe...,Released under a Creative Commons Attribution...
5232,licenses/by-nc-sa/3.0,https://www.w3.org/Submission/ccREL/,ccREL: The Creative Commons Rights Expression\...
2820,licenses/by-nc/1.0,https://www.google.com/help/legalnotices_maps/,Legal Notices for Google Maps/Google Earth and...


In [234]:
webpages_dataset_deduplicate = webpages_dataset.groupby("url").first()\
    .reset_index()
webpages_dataset_deduplicate.describe()

,url,license,contents
count,1956,1956,1956
unique,1956,39,1898
top,http://agroportal.lirmm.fr/ontologies/PCO,licenses/by/4.0,403 Forbidden 403 Forbidden nginx
freq,1,136,25


In [235]:
import dataset_sampling
license_map = dataset_sampling.get_license_map()
license_ser = pd.concat([v for v in license_map.values()])
license_ser_splits_df = license_ser.str.split("/", expand=True)
license_ser_splits_df = license_ser_splits_df.rename(
    columns = {
        0: "Tool Typing",
        1: "General Typing",
        2: "Version",
        3: "Jurisdiction"
    }
)
license_ser_splits_df["General Typing"] = license_ser_splits_df["General Typing"].str.replace("mark|zero", "publicdomain", regex=True)
license_ser_splits_df["General Typing"] = license_ser_splits_df["General Typing"].str.replace("by-nd-nc", "by-nc-nd", regex=True)
license_ser_splits_df["Version"] = license_ser_splits_df["Version"].astype(float)
license_one_hot_encoding = pd.DataFrame()
license_one_hot_encoding["by"] = license_ser_splits_df["General Typing"].str.contains("by")
license_one_hot_encoding["sa"] = license_ser_splits_df["General Typing"].str.contains("sa")
license_one_hot_encoding["nc"] = license_ser_splits_df["General Typing"].str.contains("nc")
license_one_hot_encoding["nd"] = license_ser_splits_df["General Typing"].str.contains("nd")
license_not_six_type = license_ser_splits_df["General Typing"].str.contains("by|sa|nc|nd")
license_one_hot_encoding["neither"] = ~(license_not_six_type.fillna(False))
license_df = pd.concat([license_ser, license_ser_splits_df, license_one_hot_encoding], axis = 1)\
    .rename(columns = {0: "license"})
license_df.head(6)

,license,Tool Typing,General Typing,Version,by,sa,nc,nd,neither
0,licenses/by/1.0,licenses,by,1.0,True,False,False,False,False
14,licenses/by/2.0,licenses,by,2.0,True,False,False,False,False
27,licenses/by/2.1,licenses,by,2.1,True,False,False,False,False
33,licenses/by/2.5,licenses,by,2.5,True,False,False,False,False
39,licenses/by/3.0,licenses,by,3.0,True,False,False,False,False
45,licenses/by/4.0,licenses,by,4.0,True,False,False,False,False


In [236]:
webpages_dataset_deduplicate = webpages_dataset_deduplicate.merge(license_df, on = "license")
webpages_dataset_deduplicate.sample(5)

,url,license,contents,Tool Typing,General Typing,Version,by,sa,nc,nd,neither
1613,https://research.acer.edu.au/talis/2/,licenses/by-nc-sa/3.0,"""Australian teachers and the learning environm...",licenses,by-nc-sa,3.0,True,True,True,False,False
1169,https://viva.pressbooks.pub/introbio2/chapter/...,licenses/by/2.1,6.2 Evolution of Simple Multicellularity – VCU...,licenses,by,2.1,True,False,False,False,False
650,https://mdsoar.org/handle/11603/20196,publicdomain/mark/1.0,Dynamic Linkages Among U.S. Real Estate Sector...,publicdomain,publicdomain,1.0,False,False,False,False,True
1766,https://www.ebi.ac.uk/ols/ontologies/bto,licenses/by-nd/1.0,The BRENDA Tissue Ontology (BTO) < Ontology Lo...,licenses,by-nd,1.0,True,False,False,True,False
688,https://spiral.imperial.ac.uk/handle/10044/1/5...,publicdomain/mark/1.0,Spiral: Targeting cattle for malaria eliminati...,publicdomain,publicdomain,1.0,False,False,False,False,True


In [237]:
webpages_dataset_deduplicate.groupby("General Typing").count()

,url,license,contents,Tool Typing,Version,by,sa,nc,nd,neither
General Typing,,,,,,,,,,
by,751,751,751,751,751,751,751,751,751,751
by-nc,242,242,242,242,242,242,242,242,242,242
by-nc-nd,117,117,117,117,117,117,117,117,117,117
by-nc-sa,201,201,201,201,201,201,201,201,201,201
by-nd,131,131,131,131,131,131,131,131,131,131
by-sa,280,280,280,280,280,280,280,280,280,280
publicdomain,234,234,234,234,234,234,234,234,234,234


In [238]:
#webpages_dataset_deduplicate["contents"] = webpages_dataset_deduplicate["contents"].apply(lambda x: x[:3000])

In [239]:
import string
def remove_unicodes(ser):
    return ser.map(lambda x: " ".join([c for c in x if c in string.printable]))

def has_unicodes(s, tolerance = 30):
    return np.sum([c not in string.printable for c in s]) <= tolerance

def not_well_decrypted(s, tolerance = 25):
    words = re.split(r"\s+", s)
    return np.sum([len(c) == 1 for c in words]) <= tolerance

def remove_unicodes_aggressive(df, field_name = "contents"):
    df_remove_unicode = df.loc[df[field_name].apply(has_unicodes), :]
    return df_remove_unicode


In [240]:
# Less Aggressive pruning
#webpages_dataset_deduplicate["parsed_contents"] = remove_unicodes(webpages_dataset_deduplicate["contents"])
#webpages_dataset_deduplicate.loc[1149, ["contents", "parsed_contents"]]

# More Aggressive pruning
webpages_dataset_deduplicate = remove_unicodes_aggressive(webpages_dataset_deduplicate)

In [241]:
def remove_less_than_c_chars(s, tolerance = 2):
    words = re.split(r"\s+", s)
    return " ".join([c for c in words if len(c) > tolerance])

def remove_more_than_c_chars(s, tolerance = 15):
    words = re.split(r"\s+", s)
    return " ".join([c for c in words if len(c) <= tolerance])

def remove_non_english(s):
    words = set(nltk.corpus.words.words())
    return " ".join(w for w in re.split(r"\s+", s) if w in words)

def remove_web_urls(s):
    return re.sub(r"[-a-zA-Z0-9@:%._\+~#=]{1,256}\.[a-zA-Z0-9()]{1,6}\b([-a-zA-Z0-9()@:%_\+.~#?&//=]*)", " ", s)

def clear_stopwords(s):
    return " ".join([word for word in re.split(r"\s+", s) if word.lower() not in nltk.corpus.stopwords.words('english')])

def clear_stopwords_series(ser):
    return ser.apply(clear_stopwords)

def overall_cleaning(ser):
    cleaned_ser = ser.str.lower()
    cleaned_ser = cleaned_ser.apply(remove_web_urls)
    cleaned_ser = cleaned_ser.str.replace(r'[^A-Za-z\s]', ' ', regex = True)
    cleaned_ser = cleaned_ser.str.replace(r"\s+", " ", regex = True)
    cleaned_ser = cleaned_ser.apply(clear_stopwords)
    cleaned_ser = cleaned_ser.apply(remove_less_than_c_chars)
    cleaned_ser = cleaned_ser.apply(remove_non_english)
    cleaned_ser = cleaned_ser.apply(clear_stopwords)
    #cleaned_ser = cleaned_ser.apply(remove_more_than_c_chars)
    return cleaned_ser

webpages_dataset_deduplicate["cleaned_contents"] = overall_cleaning(webpages_dataset_deduplicate["contents"])
webpages_dataset_deduplicate = webpages_dataset_deduplicate.loc[
    webpages_dataset_deduplicate["cleaned_contents"].str.len() >= 500, :
]
#webpages_dataset_deduplicate["parsed_cleaned_contents"] = clear_stopwords_series(webpages_dataset_deduplicate["cleaned_contents"])

In [242]:
for row in webpages_dataset_deduplicate["cleaned_contents"].sample(5):
    print(
        f"Entry content:\n{row[:550]}\n"
    )

Entry content:
language maintenance cultural identity heritage university home university college theses view item disabled browser site may work without language maintenance cultural identity heritage duro advisor language learning heritage culture identity word cultural competence identity heritage date publisher university work license free copy redistribute material format well remix transform build upon material long give appropriate credit original creator provide link license indicate made may use work commercial purpose must distribute identical licen

Entry content:
certainly comfort zone qualitative study dementia care educational needs study current research staff teaching learning alumni library iris certainly comfort zone qualitative study dementia care educational needs cora home college medicine health general practice general practice journal view item disabled browser site may work without item name size format description version view open certainly comfort zone quali

## Preprocessing

In [445]:
from sklearn.model_selection import train_test_split
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics import top_k_accuracy_score
from sklearn.decomposition import PCA
import re

In [244]:
def tokenize_url(url):
    return " ".join(re.split(r"[_/\.-]", re.sub(r"\d", "", url)))
tokenize_url("creativecommons.org/licenses/by-sa/4.0")

'creativecommons org licenses by sa  '

In [245]:
dataset = webpages_dataset_deduplicate.copy()
dataset['token_url'] = dataset["url"].apply(tokenize_url)
dataset["train_text"] = dataset["token_url"] + " " + dataset["cleaned_contents"]

## Modeling

In [570]:
def extract_text_features_tfidf(train, test, text_field = "train_text", svd = True):
    tfidf_vectorizer = TfidfVectorizer(
        use_idf=True, stop_words="english", max_features = 5000, sublinear_tf=True
    )
    tfidf_vectorizer.fit(train[text_field].values)
    train_vectorized = tfidf_vectorizer.transform(train[text_field].values)
    test_vectorized = tfidf_vectorizer.transform(test[text_field].values)
    if svd:
        tsvd = TruncatedSVD(n_components = 150)
        tsvd.fit(train_vectorized)
        train_vectorized = tsvd.transform(train_vectorized)
        test_vectorized = tsvd.transform(test_vectorized)
    return train_vectorized, test_vectorized, tfidf_vectorizer

In [571]:
from imblearn.over_sampling import SMOTE

In [572]:
dataset.groupby("General Typing")[["url"]].count().reset_index()["url"]

0    605
1    190
2     83
3    162
4     95
5    199
6    189
Name: url, dtype: int64

In [573]:
model_dataset = pd.DataFrame()
dataset_counts = dataset.groupby("General Typing")[["url"]].count().reset_index()["url"]
license_dict = {
        "by": 0,
        "by-nc": 1,
        "by-nc-nd": 2,
        "by-nc-sa": 3,
        "by-nd": 4,
        "by-sa": 5,
        "publicdomain": 6
}

model_dataset["train_text"], model_dataset["General Typing"] = dataset["cleaned_contents"], dataset["General Typing"]
model_dataset["General Typing"].replace(
    license_dict,
    inplace = True
)

training_set, test_set = train_test_split(model_dataset, test_size = 0.2)
Y_train = training_set["General Typing"].values
Y_test = test_set["General Typing"].values
X_train, X_test, model_vecter = extract_text_features_tfidf(
    training_set, test_set
)

smote = SMOTE(
    sampling_strategy={
        k: int(0.8 * min(round(np.mean(dataset_counts.values)), dataset_counts.iloc[k] * 1.4))
        for k in range(1, 7)} #
)
X_train, Y_train = smote.fit_resample(X_train, Y_train)


### log reg

In [574]:
model_logreg = LogisticRegression(
    verbose = 0,
    penalty = 'l2',
    solver = "liblinear",
    class_weight = "balanced",
    C = 0.05
).fit(X_train, Y_train)

In [575]:
print(
    top_k_accuracy_score(Y_train, model_logreg.predict_proba(X_train), k = 1),
    "\n",
    top_k_accuracy_score(Y_train, model_logreg.predict_proba(X_train), k = 2),
    "\n",
    top_k_accuracy_score(Y_train, model_logreg.predict_proba(X_train), k = 3),
    "\n==============\n",
    top_k_accuracy_score(Y_test, model_logreg.predict_proba(X_test), k = 1),
    "\n",
    top_k_accuracy_score(Y_test, model_logreg.predict_proba(X_test), k = 2),
    "\n",
    top_k_accuracy_score(Y_test, model_logreg.predict_proba(X_test), k = 3)
)

0.3698529411764706 
 0.6764705882352942 
 0.8242647058823529 
 0.4786885245901639 
 0.7081967213114754 
 0.8163934426229508


### SVM

In [576]:
model_svc = SVC(
    C = 0.5,
    verbose = 1,
    probability = True,
    kernel = "poly",
    degree = 1,
    class_weight = 'balanced'
).fit(X_train, Y_train)

[LibSVM]

In [577]:
print(
    top_k_accuracy_score(Y_train, model_svc.predict_proba(X_train), k = 1),
    "\n",
    top_k_accuracy_score(Y_train, model_svc.predict_proba(X_train), k = 2),
    "\n",
    top_k_accuracy_score(Y_train, model_svc.predict_proba(X_train), k = 3),
    "\n==============\n",
    top_k_accuracy_score(Y_test, model_svc.predict_proba(X_test), k = 1),
    "\n",
    top_k_accuracy_score(Y_test, model_svc.predict_proba(X_test), k = 2),
    "\n",
    top_k_accuracy_score(Y_test, model_svc.predict_proba(X_test), k = 3)
)

0.5625 
 0.7867647058823529 
 0.8985294117647059 
 0.5081967213114754 
 0.7016393442622951 
 0.7934426229508197


### Tensorflow

In [374]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
from official.nlp import optimization  # to create AdamW optimizer

import matplotlib.pyplot as plt

tf.get_logger().setLevel('ERROR')

In [375]:
tfhub_handle_preprocess = 'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/2'
tfhub_handle_encoder = 'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1'
bert_preprocess_model = hub.KerasLayer(tfhub_handle_preprocess)
bert_model = hub.KerasLayer(tfhub_handle_encoder)

In [376]:
AUTOTUNE = tf.data.AUTOTUNE
BATCH_SIZE = 16
TRAINING_RATIO = 0.7

model_dataset = dataset.loc[:, ["cleaned_contents", "General Typing"]]
model_dataset["General Typing"].replace(
    license_dict,
    inplace = True
)

model_dataset_copy = model_dataset[
    ~model_dataset["General Typing"].isin([2, 4])
]

target = model_dataset_copy.pop("General Typing")
# dataset_tensor = tf.convert_to_tensor(model_dataset, dtype=np.string_)
tf_dataset = (
    tf.data.Dataset
    .from_tensor_slices((model_dataset_copy, target))
    .shuffle(50)
)
train_dataset = tf_dataset.take(int(TRAINING_RATIO * len(tf_dataset)))
test_dataset = tf_dataset.skip(int(TRAINING_RATIO * len(tf_dataset)))
val_dataset = test_dataset.skip(
    int((1 - TRAINING_RATIO) * 0.5 * len(tf_dataset))
)
test_dataset = test_dataset.take(
    int((1 - TRAINING_RATIO) * 0.5 * len(tf_dataset))
)

train_dataset = train_dataset.batch(BATCH_SIZE)
val_dataset = val_dataset.batch(BATCH_SIZE)
test_dataset = test_dataset.batch(BATCH_SIZE)

In [377]:
def build_classifier_model():
    text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='parsed_cleaned_contents')
    preprocessing_layer = hub.KerasLayer(tfhub_handle_preprocess, name='preprocessing')
    encoder_inputs = preprocessing_layer(text_input)
    encoder = hub.KerasLayer(tfhub_handle_encoder, trainable=True, name='BERT_encoder')
    outputs = encoder(encoder_inputs)
    net = outputs['pooled_output']
    net = tf.keras.layers.Dropout(0.3)(net)
    net = tf.keras.layers.Dense(1, activation=None, name='classifier')(net)
    return tf.keras.Model(text_input, net)

In [378]:
from tensorflow_addons.metrics import F1Score

In [379]:
epochs = 10
steps_per_epoch = tf.data.experimental.cardinality(train_dataset).numpy()
num_train_steps = steps_per_epoch * epochs
num_warmup_steps = int(0.1*num_train_steps)

init_lr = 3e-5
optimizer = optimization.create_optimizer(init_lr=init_lr,
                                          num_train_steps=num_train_steps,
                                          num_warmup_steps=num_warmup_steps,
                                          optimizer_type='adamw')

In [381]:
classifier_model = build_classifier_model()
classifier_model.compile(
    optimizer=optimizer,
    loss='categorical_crossentropy',
    metrics=["accuracy", F1Score(1, average = "weighted")]
)
history = classifier_model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=epochs
)

Epoch 1/10
59/59 [==============================] - 96s 2s/step - loss: 2.8529e-07 - accuracy: 0.2742 - f1_score: 1.4106 - val_loss: 2.6426e-07 - val_accuracy: 0.2759 - val_f1_score: 1.3783
Epoch 2/10
59/59 [==============================] - 90s 2s/step - loss: 2.8491e-07 - accuracy: 0.2954 - f1_score: 1.4100 - val_loss: 2.6484e-07 - val_accuracy: 0.2611 - val_f1_score: 1.3792
Epoch 3/10
59/59 [==============================] - ETA: 0s - loss: 2.8415e-07 - accuracy: 0.2880 - f1_score: 1.4089

KeyboardInterrupt: 